# Introduction
This notebook covers uploading semantic segmentations a study level task. 

Within `NIFTI_images` are 7 MRI studies, each containing 4 different series each - flair, t1, t1ce, t2. Each series has a corresponding semantic segmentation NIfTI map within `NIFTI_segmentations`.

The following code will create the correct mapping `JSON` files required to upload 7 study level tasks with pre-segmentations for each series. 

> ❕ Please have a look at the reference formats found in `format.md`

In [2]:
import os
import json

## Create a mapping file
Because we are working with semantic segmentation annotations, each category maps directly to a single class ID. The following object maps between the class ID within your NIfTI files, and the category name of your RedBrick AI taxonomy. 

In [3]:
mapping = {
    "1": "necrosis",
    "2": "edema",
    "3": "non-enhancing tumor",
    "4": "enhancing tumor"
}
with open("mapping.json", "w+") as file:
    json.dump(mapping, file, indent=2)

## Create your items file
Next, we must create an items file that will upload your studies and corresponding segmentation maps.

In [8]:
root = "brain-brats-study"
image_dir = "NIFTI_images"
seg_dir = "NIFTI_segmentations"
items = []
for study in os.listdir(os.path.join(root, image_dir)):
    if not os.path.isdir(os.path.join(root, image_dir, study)):
        continue
    
    series = os.listdir(os.path.join(root, image_dir, study))
    items += [
        {
            "name": study,
            "items": [
                os.path.join(root, image_dir, study, file) for file in series
            ],
            "segmentations": [
                os.path.join(root, seg_dir, study, file[:-7] + "_seg.nii.gz") for file in series
            ]
        }
    ]

In [9]:
with open("items.json", "w+") as file:
    json.dump(items, file, indent=2)

## Upload data using CLI
Once you have created the mapping, and items JSON files, you can upload the data using the RedBrick AI CLI. First perform the standard set up: 

```console
$ pip install -U redbrick-sdk
$ redbrick config // Fill out org_id, api_key etc.
$ redbrick clone project_id // or use redbrick init to create a new project
$ cd project_directory
```

### Locally stored data
If your data is stored locally, the process is straightforward:

```console
$ redbrick upload --json items.json --segment-map mapping.json
```
This command prompts the CLI to upload the locally stored data/segmentations in `items.json`, and the `mapping.json` file gives information on how to map class ID's to category names

### Cloud data, local annotations
In the case your data is stored in the cloud e.g. AWS s3, and annotations are stored locally, you need to do the following:

```console
$ redbrick upload --storage <storage_id> --json items.json --segment-map mapping.json
```


The `--storage` flag will prompt RedBrick AI to fetch only the image data from `items.json` from your cloud bucket (identified by `<storage_id>`, you can find your `<storage_id>` withing the Storage Tab on the RedBrick AI platform). With this command, the segmentations will be uploaded from your local machine. 

### Cloud data, and cloud annotations
In the case both your annotations and data are stored in the cloud, e.g. AWS s3, you need to first set your projects annotation storage location to your clod bucket. This is a project wide setting, and when this is set to your cloud bucket, RedBrick will read and write annotations directly from your bucket. Note, this requires your bucket to be configured to have `GET` and `PUT` object access.

```console
$ redbrick info --set labelstorage
> Storage ID: <storage_id>
> Path prefix: path/to/label/directory
```

Once you've set your labelstorage method to your cloud bucket, you can upload the data as follows: 
```console
$ redbrick upload --storage <storage_id> --json items.json --segment-map mapping.json
```
This command will prompt RedBrick to look for your data at the paths defined in `items.json` within your bucket (defined by `<storage_id>`), and it will look for your annotation files within the bucket, and path you set in the previous command. 